Mini Project 5
Abbaspour
610398147

Welcome to the biggest mini project ever!
We're gonna train three word embedding models "Word2Vec", "GloVe" and "FastText" with first 1000 allocated documents for training these models
and also Naive Bayes classifier and Support Vector Machine classifier.
We'd like to do these with Latent Semantic Analysis (approximately, indeed!)
First of all, as always, let's import essential libraries.
The nltk attributes and punctuation from string library are imported for preprocessing the documents.
listdir from os make us be accessed to data sets.
distance from Levenshtein is used in finding and returning the most similar word exists in the vocabulary of any trained models.
Library numpy attributes and crs_matrix from scipy.sparse are imported in order to handle arrayed operations and function.
MultinomialNB from sklearn.naive_bayes is imported for creating and training Naive Bayes classifier.
CountVectorizer and TfidfVectorizer from sklearn.feature_extraction.text are imported for TF vectorizing and TF-IDF vectorizing documents, respectively.
TruncatedSVD from sklearn.decomposition is imported for Latent Semantic Analysis.
SVC from sklearn.svm is imported for creating and training Support Vector Machine classifier.
Whatever imported from sklearn.metrics is for machine learning methods evaluations.
All imported attributes from matplotlib library have functions in plotting.
All imported attributes from gensim library have functions in creating and training word embedding models for documents and words included.

In [ ]:
import nltk, itertools
from nltk.tokenize import wordpunct_tokenize
#nltk.download('stopwords')
from nltk.corpus import stopwords
from string import punctuation
from os import listdir
from Levenshtein import distance
from numpy import array, add, zeros, transpose
from scipy.sparse import csr_matrix
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.svm import SVC
from sklearn.metrics import precision_recall_curve, f1_score, precision_score, recall_score, accuracy_score, classification_report, confusion_matrix, ConfusionMatrixDisplay
from matplotlib.pyplot import plot, subplots, box, show, scatter, title, xlabel, ylabel, annotate
from matplotlib.colors import ListedColormap
from gensim.models import Word2Vec, FastText, KeyedVectors
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.models.phrases import Phrases, Phraser
from gensim.scripts.glove2word2vec import glove2word2vec

Pre proccesing function is now can only tokenize texts and removing some special useless characters included in data texts.

In [ ]:
def preProcessor(String):
    stopWords, String, Specials = stopwords.words('english'), wordpunct_tokenize(String), [").", "\'", ",", ";", ".<", "br", "/><", "/>"] # Case Folding & Tokenisations
    return [Token.lower() for Token in String if Token not in punctuation and Token.lower() not in stopWords and Token not in Specials]

This function returns the most similar word exists in the vocabulary of any trained models. It handles key error in using the vector of words in documents which doesn't exist in vocabulary of any trained models.

In [ ]:
def Approx(unknownWord, Vocabulary):
    # Listing similar words in the vocabulary to the unknown words.
    listCounter = [distance(unknownWord, Word) for Word in Vocabulary]

    # Retruning most similar word exists.
    return Vocabulary[listCounter.index(min(listCounter))]

This function is defined for SVD truncating the vectors of documents.
The motivation of defining this function was to use it in the way of classify them with SVM, as we'll see, making document vectors and truncating them wouldn't be used.
Despite description above, it's defined just in case.
The documents topics are separated into two topics: "Like" and "Dislike".

In [ ]:
def term_doc_mat_LSA():
    Topics = {0: "Like", 1: "Disike"}
    LSA = LSI.fit_transform(tfidf.fit_transform(trainDocs))
    print(f"\nSigma:\n{LSI.singular_values_}\n") #Sigmas
    print(f"V^T:\n{LSI.components_.T}\n")        #VT
    Terms = tfidf.get_feature_names_out()
    for Index, Component in enumerate(LSI.components_):
        print(f"{Topic[Index]}: {list(dict(sorted(zip(Terms, Component), key = lambda t: t[1], reverse = True)[:5]).keys())}\n")

    print(LSI.explained_variance_ratio_)
    plot(range(2), LSI.explained_variance_ratio_ * 100)
    xlabel("Topic Number")
    ylabel("% explained")
    title("SVD dropoff")
    show()  # show first chart

(Optional) This was defined for the prior fifth mini project which was truncated document vectors in any word embedding form by SVD in order to be used in Latent Semantic Analysis. The idea of making matrix of document vectors for SVD tranformation is from a paper attached to the zip file.

In [ ]:
def documentLSA():
    trainDocs_tf, trainDocs_tfidf, testDocs_tf = tfVectorizer.fit_transform(trainDocs), tfidfVectorizer.fit_transform(trainDocs), tfVectorizer.transform(testDocs)
    tf_doc_term_matrix, tfidf_doc_term_matrix = trainDocs_tf.toarray(), trainDocs_tfidf.toarray() # Ordered document-term by tf.
    sortedWords = sorted(list(set([Word for Sentence in Sentences for Word in Sentence])))[:tf_doc_term_matrix.shape[1]] # Dimension reduction for creating document vector matrix. (A logical error occures that latter words are not considered.)
    term_doc_matrix, wordEmbeddings, validResponses = {"TF": "tf term-document matrix", "IDF": "tf-idf term-document matrix"}, {"W2V": "Word2Vec", "GLV": "GloVe", "FT": "FastText"}, ["TF", "IDF"]
    while True:
        term_doc_matrix_form = input("Tf term-document matrix or tf-idf term-document matrix? <Tf, Idf> ").upper()
        if term_doc_matrix_form in validResponses:
            print(f"\nYou chose {term_doc_matrix[term_doc_matrix_form]} matrix form.")
            break
        print("\nSelect one, dude!")

    validResponses = ["W2V", "GLV", "FT"]
    while True:
        word_embedding_technique = input("Word2Vec, GloVe or FastText? <W2V, GLV, FT> ").upper()
        if word_embedding_technique in validResponses:
            print(f"\nYou chose {wordEmbeddings[word_embedding_technique]} word embedding technique.")
            break
        print("\nSelect one, dude!")

    if term_doc_matrix_form == "TF":
        # transpose(document-term matrix * word-embedded word vector matrix) = embedded word vector-document matrix.
        if word_embedding_technique == "W2V": 
            return SVD.fit_transform(csr_matrix(transpose(tf_doc_term_matrix.dot(WordVec(Sentences)))))

        elif word_embedding_technique == "GLV":
            return SVD.fit_transform(csr_matrix(transpose(tf_doc_term_matrix.dot(GloVe(Sentences)))))
            
        return SVD.fit_transform(csr_matrix(transpose(tf_doc_term_matrix.dot(fastText(Sentences)))))

    if word_embedding_technique == "W2V":
        return SVD.fit_transform(csr_matrix(transpose(tfidf_doc_term_matrix.dot(WordVec(Sentences)))))

    elif word_embedding_technique == "GLV":
        return SVD.fit_transform(csr_matrix(transpose(tfidf_doc_term_matrix.dot(GloVe(Sentences)))))
            
    return SVD.fit_transform(csr_matrix(transpose(tfidf_doc_term_matrix.dot(fastText(Sentences)))))

This is used for achiving document vectors by means of Word2Vec word embedding approach.

In [ ]:
def DocVec():
    taggedData = [TaggedDocument(words = trainDoc.split(), tags = [str(Tag)]) for Tag, trainDoc in enumerate(trainDocs)]
    # train the Doc2vec model
    Model = Doc2Vec(vector_size = 300, min_count = 2, epochs = 50)
    Model.build_vocab(taggedData)
    Model.train(taggedData, total_examples = Model.corpus_count, epochs = Model.epochs)

    return [Model.infer_vector(trainDoc.split()) for trainDoc in trainDocs]

This function is used for creating and training a model with one of two selective Word2Vec word embedding approaches "Continuous Bag of Words" and "Skipgram".

In [ ]:
def WordVec(Sentences):
    validResponses = ["CBOW", "SG"]
    while True:
        modelMethod = input("CBOW or Skip-gram? <CBOW, Sg> ").upper()
        if modelMethod in validResponses:
            print("Alright!")
            break
        print("Select one, dude!")
    
    if modelMethod == "CBOW":
        # Create CBOW model
        Model = Word2Vec(min_count = 5, vector_size = 300, window = 5)
    else:
        # Create Skip-gram model(sg = 1)
        Model = Word2Vec(min_count = 5, vector_size = 300, window = 5, sg = 1)

    Model.build_vocab(Sentences, progress_per = 1000)

    # train on our data
    Model.train(Sentences, total_examples = len(Sentences), epochs = 100)

    return Model.wv

This function is used for creating and training a model with GloVe word embedding approach.
It uses Stanford pretrained Glove model and then, update it with own dataset for training.

In [ ]:
def GloVe(Sentences):
    # Stanford pretrained
    # Load GloVe embeddings using Gensim
    gloVectors = KeyedVectors.load_word2vec_format('glove.6B.100d.txt', binary = False, no_header = True) # New version!

    # build a toy model to update with
    Model = Word2Vec(vector_size = 300, min_count = 5)
    Model.build_vocab(Sentences)
    numberExamples = Model.corpus_count

    # add GloVe's vocabulary & weights.
    Model.build_vocab([list(gloVectors.index_to_key)], update = True)

    # train on our data
    Model.train(Sentences, total_examples = numberExamples, epochs = Model.epochs)

    return Model.wv

This function is used for creating and training a model with FastText word embedding approach.

In [ ]:
def fastText(Sentences):
    phraseSentences = Phrases(Sentences, min_count = 30, progress_per = 10000)[Sentences]
    Model = FastText(vector_size = 300, window = 5, min_count = 5, workers = 4, min_n = 1, max_n = 4)
    
    #Building Vocabulary
    Model.build_vocab(phraseSentences)

    # train on our data
    Model.train(phraseSentences, total_examples = len(Sentences), epochs = 100)
    
    return Model.wv

Here's the function training SVM with or without Latent Semantic Analysis.
Function 'Vectorizer' uses vectors of each word in any word embedding model and prepares them for SVM classifying.
Function 'Suppvec' train Support Vector Machine classifier by means of word vectors by Vectorizer function.

In [ ]:
def Vectorizer(Sentence, WVs, LSA):
    if LSA:
        wordsVecs = LSI.fit_transform(csr_matrix([WVs[Word] for Word in Sentence if Word in WVs]))
    else:
        wordsVecs = [WVs[Word] for Word in Sentence if Word in WVs]
        
    if len(wordsVecs) == 0:
        return zeros(300)
    
    wordsVecs = array(wordsVecs)
    return wordsVecs.mean(axis = 0)

def SuppVec(WVs, LSA):
    transformedTrain = array([Vectorizer(Sentence, WVs, LSA) for Sentence in Sentences])
    transformedTest = array([Vectorizer(Sentence, WVs, LSA) for Sentence in Sentences])

    # Train a classification model
    SVM.fit(transformedTrain, trainLabels)

    predictLabels = SVM.predict(transformedTest)
    print('Accuracy: ', accuracy_score(testLabels, predictLabels))
    print('Precision: ', precision_score(testLabels, predictLabels, pos_label = 1))
    print('Recall: ', recall_score(testLabels, predictLabels, pos_label = 1))
    print('F1 score: ', f1_score(testLabels, predictLabels, pos_label = 1))
    
    # Generate the classification report.
    print(classification_report(testLabels, predictLabels))

    # Generate scatter plot for training data
    scatter(transformedTrain[:,0], transformedTrain[:,1])
    title('Linearly separable data')
    xlabel('X1')
    ylabel('X2')
    show()
    
    # Get support vectors
    supportVectors = SVM.support_vectors_

    # Visualize support vectors
    ConfusionMatrixDisplay(confusion_matrix = confusion_matrix(testLabels, predictLabels), display_labels = ["Positive", "Negative"]).plot()
    scatter(transformedTrain[:,0], transformedTrain[:,1])
    scatter(supportVectors[:,0], supportVectors[:,1], color = 'red')
    title('Linearly separable data with support vectors')
    xlabel('X1')
    ylabel('X2')
    show()

This function trains Naive Bayes classifier with (OPTIONAL) or without TF-IDF vectorizing.

In [ ]:
def NB(tfidf):
    # Training by Naive Bayes classifier method.
    if tfidf:
        # tf-idf vectorizing approach.
        transformedTrain, transformedTest = tfVectorizer.fit_transform(trainDocs), tfVectorizer.transform(testDocs)
    else:
        vectorizerFit = tfVectorizer.fit(trainDocs)
        transformedTrain, transformedTest = vectorizerFit.transform(trainDocs), vectorizerFit.transform(testDocs)

    NBClassifier.fit(transformedTrain, trainLabels)

    # Reporting accuracy.
    predictLabels = NBClassifier.predict(transformedTest)
    print(f"Harmonic mean (F1 score) for Naive Bayes is {f1_score(testLabels, predictLabels)}\n")
    print(f"Accuracy score for Naive Bayes is {accuracy_score(testLabels, predictLabels)}\n")
    
    # Comprehensive classification report.
    print(f"Classification Report:\n{classification_report(testLabels, predictLabels)}")
    ConfusionMatrixDisplay(confusion_matrix = confusion_matrix(testLabels, predictLabels), display_labels = ["Positive", "Negative"]).plot()

    # Plotting Precision-Recall Curve
    Precision, Recall, Threshold = precision_recall_curve(testLabels, NBClassifier.predict_proba(transformedTest)[:,1])
    fig, ax = subplots(figsize = (6,6))
    ax.plot(Recall, Precision, label = "Naive Bayes Classification", color = "firebrick")
    ax.set_title("Precision-Recall Curve")
    ax.set_xlabel("Recall")
    ax.set_ylabel("Precision")
    box(False)
    ax.legend()
    show()

Now here's is allocating memories for some variables and creating preprocessed train and test documents with their corresponding labels.

In [ ]:
# Globals!
tfVectorizer, tfidf, NBClassifier, wordEmbeddings, docEmbeddings, SVM, LSI, posTrain, negTrain, posTest, negTest, trainDocs, testDocs, trainLabels, testLabels, Sentences = CountVectorizer(), TfidfVectorizer(), MultinomialNB(), {WordVec: "Word2Vec", GloVe: "GloVe", fastText: "FastText"}, ["DV", "WV"], SVC(C = 1.0, kernel = 'linear', degree = 3, gamma = 'auto'), TruncatedSVD(n_components = 2, algorithm="arpack"), listdir("train/pos"), listdir("train/neg"), listdir("test/pos"), listdir("test/neg"), [], [], [], [], []

# Bound the number of trained and test data by 5000. Label '1' for positive and label '0' for negative.
for Index in range(1000):
    # Train data
    Pos, Neg = preProcessor(open(f"train/pos/{posTrain[Index]}", "r").read()), preProcessor(open(f"train/neg/{negTrain[Index]}", "r").read())
    Sentences += [Pos, Neg]

    trainDocs += [" ".join(Pos), " ".join(Neg)]
    trainLabels += [1, 0]

    # Test data
    testDocs += [" ".join(preProcessor(open(f"test/pos/{posTest[Index]}", "r").read())), " ".join(preProcessor(open(f"test/neg/{negTest[Index]}", "r").read()))]
    testLabels += [1, 0]

Calling Latent Semantic Analysis term-document form and two forms of Naive Bayes classifying.

In [ ]:
# Let's start!
print("A good approximation for term-document with tf-idf metric is:")
term_doc_mat_LSA()
print("\nHere's Naive Bayes classifiying results trained by train documents and labels and evaluated by being test on test documents and labels.\n")
NB(False)
print("And with tf-idf approach:")
NB(True)

Now it's time to create document vectors in two ways (Doc2Vec or summation of vectors of words in document respectively)(optional and useless so it became commented for proving that is made and made a requirement for this project) and check two forms of SVM classifying.

In [ ]:
for wordEmbedding in wordEmbeddings: # Check evaluations for each wordembedding method
    vectorWords, embeddingTechnique = wordEmbedding(Sentences), wordEmbeddings[wordEmbedding]
    print(f"\nWith {embeddingTechnique} word embedding technique.")
    while True:
        docEmbedding = input("\nWanna transform documnets to vectors strictly or with word embedding vectors? <DV, WV> ").upper()
        if docEmbedding in docEmbeddings:
            print("\nAlright")
            break
        print("\nChoose one, boy!")

    if docEmbedding == "DV":
        vectorsDocs = DocVec()
    else:
        Vocabulary, Starter, vectorsDocs = list(vectorWords.key_to_index.keys()), array([0] * 300), []
        for Document in Sentences:
            documentVector = Starter
            for Word in Document:
                try:
                    documentVector = add(documentVector, vectorWords[Word])
                except KeyError:
                    documentVector = add(documentVector, vectorWords[Approx(Word, Vocabulary)])

            vectorsDocs.append(documentVector)

    print(f"\nHere's Raw Support Vector Machine classifying with {embeddingTechnique} form embedded word vectors.")
    SuppVec(vectorWords, False)
    print(f"\nAnd now, SVM according to Latent Semantic Analysis!")
    SuppVec(vectorWords, True)

Evaluations:
According to classification report method, Accuracy, Precision, Recall, F1 score and Precision-Recall curve in both Naive Bayes approaches become the same.
According to classification report method, Accuracy, Precision, Recall and F1 score in highest to lowest word embedding method order is:
Word2Vec > FastText >> GloVe

Advantages of Word2vec:
The idea is very intuitive, which transforms the unlabled raw corpus into labeled data (by mapping the target word to its context word), and learns the representation of words in a classification task.
The data can be fed into the model in an online way and needs little preprocessing, thus requires little memory.
The mapping between the target word to its context word implicitly embeds the sub-linear relationship into the vector space of words, so that relationships like “king:man as queen:woman” can be infered by word vectors.
It is simple for a freshman to understand the principle and do implementation.

Disadvantages of Word2vec:
The sub-linear relationships are not explicitly defined. There is little theoretical support behind such characteristic.
The model could be very difficult to train if use the softmax function, since the number of categories is too large (the size of vocabulary). Though approxination algorithms like negative sampling (NEG) and hierarchical softmax (HS) are proposed to address the issue, other problems happen. For example, the word vectors by NEG are not distributed uniformally, they are located within a cone in the vector space hence the vector space is not sufficiently utilized.


Advantages of Glove:
The goal of Glove is very straightforward, i.e., to enforce the word vectors to capture sub-linear relationships in the vector space. Thus, it proves to perform better than Word2vec in the word analogoy tasks.
Glove adds some more practical meaning into word vectors by considering the relationships between word pair and word pair rather than word and word.
Glove gives lower weight for highly frequent word pairs so as to prevent the meaningless stop words like “the”, “an” will not dominate the training progress.

Disadvantages of Glove:
The model is trained on the co-occurrence matrix of words, which takes a lot of memory for storage. Especially, if you change the hyper-parameters related to the co-occurrence matrix, you have to reconstruct the matrix again, which is very time-consuming.

Both Word2vec and Glove do not solve the problems like:
How to learn the representation for out-of-vocabulary words.
How to separate some opposite word pairs. For example, “good” and “bad” are usually located very close to each other in the vector space, which may limit the performance of word vectors in NLP tasks like sentiment analysis.

FastText is generally considered faster than Word2vec because of its use of subword information. FastText represents words as bags of character n-grams, which allows it to capture information about morphology and subword units. This approach enables FastText to handle out-of-vocabulary words better than Word2vec and generally results in faster training times. However, the actual speed comparison can depend on various factors such as the size of the dataset, the specific implementation, and the hardware used for training.
With any word embedding methods, applying Latent Semantic Analysis decreases evaluating metrics of SVM classification still in the same order above.
(Probably due to using approximation and truncating)

The main challenge which still exists is how to use raw text files in continous operating and function calling being done without geeting error.
Other challanges was training GloVe on own data and attribute errors from gensim library which has been upgraded from verison 3.0 to 4.0.
 